In [ ]:
# %pip install transformers

In [55]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

import warnings
warnings.filterwarnings('ignore')

import transformers

from transformers import GPT2Tokenizer

import textwrap

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [5]:
with open('IT_Advices_Proverbs.txt', encoding='utf8') as f:
    text = f.read()
    text = text.split('\n')

random.shuffle(text)
print(text)


['У неправедных судей много затей', 'Разработка через тестирование поощряет простой дизайн и внушает уверенность.', 'За грехи мука, за воровство кнут', 'Простота реализации важнее простоты интерфейса. ', 'Полнотой можно жертвовать в пользу остальных качеств и обязательно нужно жертвовать, если она мешает простоте. ', 'Возможности, которые не описаны в требованиях к системе, просто не должны реализовываться.', 'Мне что до кого? Было б нам хорошо', 'Программные сущности не должны зависеть от методов, которые они не используют.', 'Задавание вопроса не должно менять ответ.', 'Наряд соколий, а походка воронья', 'Если назначение элемента или сочетания неясно, то его поведение должно быть наиболее ожидаемым со стороны пользователя.', 'Не спеши, коза, все волки твои будут', 'Функции, которые используют базовый тип, должны иметь возможность использовать подтипы базового типа, не зная об этом.', 'Ближняя — с зароком, а дальняя — с двумя', 'Шилом моря не нагреешь', 'Не повторяйся.', 'Правда светл

In [6]:
train = []
test = []
max_length = 0
index_max_length = 0

for idx, value in enumerate(text):
  tokens = tokenizer.encode(value, add_special_tokens=True)
  tokens = np.array(tokens)

  curr_len = len(tokens)
  if curr_len >= max_length:
    max_length = curr_len
    index_max_length = idx

  if idx <= (len(text) * .8):
    train.append(tokens)

  else:
    test.append(tokens)

train = np.array(train)
test = np.array(test)

print('len(train), len(test): ', len(train), len(test))
print('max_length, index_max_length: ', max_length, index_max_length)


len(train), len(test):  141 34
max_length, index_max_length:  36 105


In [9]:
def Padding(review_int, seq_len):
    '''
    Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(review_int), seq_len), dtype = int)
    for i, review in enumerate(review_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            # print(len(zeros))
            # print(review.shape)
            # new = np.array(zeros) + review
            new = np.append(zeros, review)

        features[i, :] = np.array(new)
            
    return features

train = Padding(train, max_length)
test = Padding(test, max_length)

print(train.shape)
print(train[105, :])

(141, 36)
[ 6161   387 26198   281 14311 26005    16   670   322   898 33125  7372
   290  1129  4419    16  1655  9904  1882 16895  6584    16  1033 14308
   539  3003    16  9258  4587   289 11273    16   289  3531  8266    18]


In [ ]:
from transformers import GPT2LMHeadModel, AdamW

model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

In [100]:
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

batch_size = 8
epochs = 100

n_train = len(train)//batch_size + 1
n_test = len(test)//batch_size + 1
print(n_train, n_test)

total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

18 5


In [ ]:
# range(n_train)

In [101]:
def prep_tensors(x, i, batch_size=batch_size):
    start_idx = i*batch_size
    end_idx = start_idx + batch_size
    batch_ids = x[start_idx: end_idx]
    batch_ids = torch.tensor(batch_ids).to(device)
    return torch.tensor(batch_ids).to(device)

preped = prep_tensors(train, 17)
print('preped shape: ', preped.shape)

preped shape:  torch.Size([5, 36])


In [102]:

for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()

    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                             token_type_ids=None, 
                            #  attention_mask=batch_mask,
                             labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)

epoch 1/100 : training


  0%|          | 0/18 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 2/100 : training


...

  0%|          | 0/5 [00:00<?, ?it/s]

epoch 100/100 : training


  0%|          | 0/18 [00:00<?, ?it/s]

epoch {epoch}/{epochs} : validation


  0%|          | 0/5 [00:00<?, ?it/s]

In [92]:
# Model without training

prompt = 'Не повторяй'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=40,
    num_beams=5,
    do_sample=True,
    temperature=.7,
    top_k=10,
    top_p=0.95,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
  wraped = textwrap.fill(tokenizer.decode(out_), 120)
  wraped = wraped.replace("  ", " ")
  arr = wraped.split('.')
  arr.pop()
  final_out_text = '.'.join(arr) + '.'
  print(final_out_text, end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Не повторяй моих ошибок.  — Да, — согласился я, и в этот момент в дверь постучали. Я обернулся и увидел, что на пороге
стоит тот самый человек, который
------------------
Не повторяй моих ошибок.  — Нет, — сказал я, не зная, что сказать. Я не знал, как поступить, и не хотел, чтобы кто-то
знал.
------------------
Не повторяй моих ошибок.  — Да, — согласился я, и в этот момент в дверь постучали. Я обернулся и увидел, что на пороге
стоит мой друг. Он был
------------------
Не повторяй моих ошибок.  — Я не хочу, чтобы ты повторяла мои ошибки, — сказал он, и в его голосе прозвучала мольба,
которую он не мог скрыть
------------------
Не повторяй моих ошибок.  — Да, — согласился я, и мне показалось, что я слышу его голос, но я не мог разобрать его слов,
потому что он был не
------------------
Не повторяй моих ошибок.  — Да, — согласился я, и в этот момент в дверь постучали. Я обернулся и увидел, что на пороге
стоит тот самый человек, который
------------------
Не повторяй моих ошибок.  — Да, 

In [118]:
# Trained model

prompt = 'Дизайн должен быть'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)

out = model.generate(
    input_ids=prompt,
    max_length=40,
    num_beams=5,
    do_sample=True,
    temperature=.7,
    top_k=10,
    top_p=0.95,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()

for out_ in out:
  wraped = textwrap.fill(tokenizer.decode(out_), 120)
  wraped = wraped.replace("  ", " ")
  arr = wraped.split('.')
  arr.pop()
  final_out_text = '.'.join(arr) + '.'
  print(final_out_text, end='\n------------------\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми, а не усложненными.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми, а не усложненными.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми, а не усложненными.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми, а не усложненными.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми, а не усложненными.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми, а не усложненными.
------------------
Дизайн должен быть правильным во всех видимых проявлениях. Детали должны быть простыми, а не усложненными.
------------------


In [ ]:
def generate(prompt, len_gen=20, temperature=.7):
    generated = tokenizer.encode(prompt)
    context = torch.tensor([generated]).to(device)
    past = None

    for i in tqdm(range(len_gen)):
        output, past = model(context, past_key_values=past).values()
        # token = torch.argmax(output[..., -1, :], dim=-1)
        output = output / temperature
        token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
        generated += token.tolist()
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    return sequence

In [ ]:
prompt = 'Целою толпой вышли'
print(textwrap.fill(generate(prompt, 200, temperature=.7), 120))